In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

from numpy.lib.recfunctions import drop_fields
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy import stats

# Generate complex e-commerce dataset with multiple related tables
np.random.seed(42)

# 1. Generate Customer Data
def generate_customer_data(num_customers):
    return pd.DataFrame({
        'customer_id': range(1, num_customers + 1),
        'join_date': pd.date_range(start='2023-01-01', periods=num_customers),
        'age': np.random.normal(35, 12, num_customers).astype(int),
        'income': np.random.normal(60000, 20000, num_customers).round(2),
        'country': np.random.choice(['USA', 'UK', 'Canada', 'Australia', 'Germany'], num_customers),
        'customer_segment': np.random.choice(['Premium', 'Standard', 'Basic'], num_customers),
        'subscription_type': np.random.choice(['Monthly', 'Annual', 'Pay-as-you-go'], num_customers),
        'churn_risk_score': np.random.uniform(0, 1, num_customers).round(2)
    })

# 2. Generate Product Data
def generate_product_data(num_products):
    categories = ['Electronics', 'Clothing', 'Books', 'Home', 'Sports']
    subcategories = {
        'Electronics': ['Phones', 'Laptops', 'Accessories'],
        'Clothing': ['Shirts', 'Pants', 'Dresses'],
        'Books': ['Fiction', 'Non-Fiction', 'Educational'],
        'Home': ['Kitchen', 'Furniture', 'Decor'],
        'Sports': ['Equipment', 'Clothing', 'Accessories']
    }
    
    products = []
    for i in range(num_products):
        category = np.random.choice(categories)
        products.append({
            'product_id': i + 1,
            'category': category,
            'subcategory': np.random.choice(subcategories[category]),
            'price': np.random.uniform(10, 1000, 1)[0].round(2),
            'manufacturing_cost': np.random.uniform(5, 500, 1)[0].round(2),
            'stock_level': np.random.randint(0, 1000),
            'supplier_id': np.random.randint(1, 21),
            'reorder_point': np.random.randint(10, 100),
            'rating': np.random.uniform(1, 5, 1)[0].round(1)
        })
    return pd.DataFrame(products)

# 3. Generate Transaction Data
def generate_transaction_data(num_transactions, customers_df, products_df):
    transactions = []
    for i in range(num_transactions):
        customer_id = np.random.choice(customers_df['customer_id'])
        num_items = np.random.randint(1, 6)
        
        for _ in range(num_items):
            product_id = np.random.choice(products_df['product_id'])
            product_price = products_df.loc[products_df['product_id'] == product_id, 'price'].iloc[0]
            
            transactions.append({
                'transaction_id': f"T{i+1}_{_+1}",
                'customer_id': customer_id,
                'product_id': product_id,
                'date': pd.Timestamp('2024-01-01') + pd.Timedelta(days=np.random.randint(0, 365)),
                'quantity': np.random.randint(1, 5),
                'unit_price': product_price,
                'discount': np.random.choice([0, 0.1, 0.2, 0.3], p=[0.7, 0.1, 0.1, 0.1]),
                'payment_method': np.random.choice(['Credit Card', 'PayPal', 'Bank Transfer']),
                'shipping_cost': np.random.uniform(5, 20, 1)[0].round(2)
            })
    return pd.DataFrame(transactions)

# 4. Generate Marketing Campaign Data
def generate_marketing_data(num_campaigns):
    return pd.DataFrame({
        'campaign_id': range(1, num_campaigns + 1),
        'campaign_name': [f'Campaign_{i}' for i in range(1, num_campaigns + 1)],
        'start_date': pd.date_range(start='2024-01-01', periods=num_campaigns),
        'end_date': pd.date_range(start='2024-02-01', periods=num_campaigns),
        'channel': np.random.choice(['Email', 'Social Media', 'Display Ads', 'Search'], num_campaigns),
        'budget': np.random.uniform(1000, 10000, num_campaigns).round(2),
        'target_segment': np.random.choice(['Premium', 'Standard', 'Basic'], num_campaigns),
        'conversion_rate': np.random.uniform(0.01, 0.15, num_campaigns).round(3)
    })


In [27]:
# Generate all datasets
customers = generate_customer_data(1000)
products = generate_product_data(200)
transactions = generate_transaction_data(5000, customers, products)
marketing = generate_marketing_data(50)

# Save datasets
customers.to_csv('customers.csv', index=False)
products.to_csv('products.csv', index=False)
transactions.to_csv('transactions.csv', index=False)
marketing.to_csv('marketing.csv', index=False)


## Assumed Definitions
Income is assumed to be the total spend for each customer with the company. 

In [28]:
customers

,customer_id,join_date,age,income,country,customer_segment,subscription_type,churn_risk_score
0,1,2023-01-01,40,87987.11,Canada,Premium,Annual,0.30
1,2,2023-01-02,33,78492.67,Canada,Basic,Annual,0.14
2,3,2023-01-03,42,61192.61,Canada,Premium,Monthly,0.61
3,4,2023-01-04,53,47061.26,Australia,Basic,Annual,0.31
4,5,2023-01-05,32,73964.47,Germany,Premium,Annual,0.33
...,...,...,...,...,...,...,...,...
995,996,2025-09-22,31,81403.00,UK,Premium,Monthly,0.66
996,997,2025-09-23,56,59469.57,UK,Premium,Monthly,0.81
997,998,2025-09-24,42,42362.51,Germany,Standard,Pay-as-you-go,0.67
998,999,2025-09-25,28,56738.66,Germany,Basic,Pay-as-you-go,0.50


In [29]:
mean_segment_income = customers.groupby(['customer_segment', 'country'])['income'].mean()
mean_segment_income

customer_segment  country  
Basic             Australia    66163.174590
                  Canada       58954.587385
                  Germany      58811.022667
                  UK           60107.160882
                  USA          58708.471176
Premium           Australia    59714.016885
                  Canada       63143.369143
                  Germany      61094.630000
                  UK           64054.217547
                  USA          59632.731111
Standard          Australia    64630.091642
                  Canada       59818.409254
                  Germany      58836.174688
                  UK           66414.593239
                  USA          62673.937581
Name: income, dtype: float64

In [30]:
products

,product_id,category,subcategory,price,manufacturing_cost,stock_level,supplier_id,reorder_point,rating
0,1,Home,Kitchen,28.98,218.83,116,20,12,3.8
1,2,Books,Non-Fiction,625.01,28.80,844,1,33,1.6
2,3,Books,Educational,649.47,120.28,911,15,20,4.3
3,4,Clothing,Dresses,912.52,464.37,396,10,84,2.2
4,5,Electronics,Laptops,64.89,417.35,38,3,85,2.1
...,...,...,...,...,...,...,...,...,...
195,196,Home,Decor,218.17,285.11,835,11,86,3.6
196,197,Home,Kitchen,701.09,320.88,613,10,86,3.0
197,198,Clothing,Dresses,62.92,225.62,76,2,19,4.0
198,199,Sports,Clothing,597.05,271.15,367,20,24,3.3


In [31]:
transactions

,transaction_id,customer_id,product_id,date,quantity,unit_price,discount,payment_method,shipping_cost
0,T1_1,224,8,2024-08-31,3,113.61,0.0,PayPal,13.11
1,T1_2,224,177,2024-03-03,2,691.47,0.3,Bank Transfer,9.79
2,T1_3,224,30,2024-11-27,3,946.04,0.2,PayPal,15.37
3,T2_1,606,45,2024-11-13,3,744.60,0.0,PayPal,14.03
4,T3_1,230,172,2024-03-15,1,605.81,0.1,Credit Card,10.30
...,...,...,...,...,...,...,...,...,...
14954,T4999_2,981,27,2024-07-31,3,815.47,0.0,PayPal,12.47
14955,T4999_3,981,57,2024-07-01,3,110.15,0.0,Credit Card,19.93
14956,T4999_4,981,75,2024-04-11,4,104.30,0.0,PayPal,12.43
14957,T4999_5,981,85,2024-08-03,2,406.96,0.0,Credit Card,18.50


In [32]:
customer_transactions = pd.merge(customers, transactions, on='customer_id')
customer_transactions

,customer_id,join_date,age,income,country,customer_segment,subscription_type,churn_risk_score,transaction_id,product_id,date,quantity,unit_price,discount,payment_method,shipping_cost
0,1,2023-01-01,40,87987.11,Canada,Premium,Annual,0.30,T366_1,7,2024-11-27,4,946.26,0.0,PayPal,15.20
1,1,2023-01-01,40,87987.11,Canada,Premium,Annual,0.30,T366_2,29,2024-04-04,2,960.15,0.0,PayPal,10.24
2,1,2023-01-01,40,87987.11,Canada,Premium,Annual,0.30,T366_3,41,2024-05-02,3,973.12,0.0,Bank Transfer,16.93
3,1,2023-01-01,40,87987.11,Canada,Premium,Annual,0.30,T2523_1,194,2024-12-29,1,803.15,0.2,Credit Card,11.28
4,2,2023-01-02,33,78492.67,Canada,Basic,Annual,0.14,T1965_1,36,2024-10-20,2,915.69,0.0,Bank Transfer,18.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14954,1000,2025-09-26,41,45101.95,UK,Standard,Monthly,0.95,T2604_1,185,2024-09-20,1,603.02,0.2,Credit Card,18.88
14955,1000,2025-09-26,41,45101.95,UK,Standard,Monthly,0.95,T2604_2,73,2024-08-01,3,86.71,0.1,Bank Transfer,7.90
14956,1000,2025-09-26,41,45101.95,UK,Standard,Monthly,0.95,T2604_3,184,2024-10-30,2,175.28,0.0,PayPal,19.31
14957,1000,2025-09-26,41,45101.95,UK,Standard,Monthly,0.95,T2604_4,7,2024-07-13,4,946.26,0.0,PayPal,6.20


In [33]:
customer_transactions['net_value'] = (customer_transactions['quantity'] * customer_transactions['unit_price']) * (1 - customer_transactions['discount'])

In [34]:
customer_transactions.sort_values('net_value').head(10)

,customer_id,join_date,age,income,country,customer_segment,subscription_type,churn_risk_score,transaction_id,product_id,date,quantity,unit_price,discount,payment_method,shipping_cost,net_value
11610,782,2025-02-20,18,58337.89,Canada,Standard,Pay-as-you-go,0.76,T2701_1,35,2024-03-11,1,10.74,0.3,Credit Card,16.26,7.518
13428,904,2025-06-22,50,67392.84,Germany,Standard,Annual,0.51,T4601_2,35,2024-10-07,1,10.74,0.3,Bank Transfer,16.87,7.518
6007,389,2024-01-24,46,60640.08,Australia,Standard,Monthly,0.07,T4095_4,35,2024-08-26,1,10.74,0.3,Credit Card,14.20,7.518
8756,578,2024-07-31,44,83294.79,Australia,Standard,Monthly,0.94,T4443_3,35,2024-08-04,1,10.74,0.2,Bank Transfer,16.19,8.592
14295,957,2025-08-14,39,17286.52,USA,Basic,Monthly,0.53,T2225_4,35,2024-08-24,1,10.74,0.2,PayPal,7.15,8.592
8913,587,2024-08-09,29,63528.83,Canada,Basic,Annual,0.03,T1196_2,35,2024-04-22,1,10.74,0.2,Credit Card,17.55,8.592
9611,635,2024-09-26,22,53304.49,Germany,Standard,Pay-as-you-go,0.99,T4730_2,35,2024-11-01,1,10.74,0.1,Credit Card,11.49,9.666
1777,119,2023-04-29,48,64021.98,Canada,Basic,Monthly,0.54,T3043_1,35,2024-10-18,1,10.74,0.0,Bank Transfer,10.94,10.740
7940,522,2024-06-05,41,71641.95,Canada,Premium,Monthly,0.92,T3171_1,35,2024-01-04,1,10.74,0.0,Credit Card,13.19,10.740
5538,359,2023-12-25,38,30421.77,USA,Standard,Annual,0.79,T2234_4,35,2024-09-30,1,10.74,0.0,Credit Card,8.27,10.740


In [39]:
customer_transactions_mean = customer_transactions.groupby('customer_id')['net_value'].mean()
customer_transactions_mean

customer_id
1       2316.805000
2       1077.855111
3       1388.160800
4        916.542773
5       1550.726667
           ...     
996     1559.585000
997     1247.924750
998     1197.208478
999     1309.337917
1000    1072.994889
Name: net_value, Length: 998, dtype: float64

In [57]:
customer_payment_methods = customer_transactions['payment_method'].value_counts()
customer_payment_methods

payment_method
PayPal           5006
Bank Transfer    4988
Credit Card      4965
Name: count, dtype: int64

In [58]:
customer_transactions['country'].value_counts()

country
USA          3245
Germany      3054
Canada       3013
Australia    2884
UK           2763
Name: count, dtype: int64

In [59]:
customer_transactions['customer_segment'].value_counts()

customer_segment
Basic       5281
Standard    4968
Premium     4710
Name: count, dtype: int64

In [60]:
customer_transactions['subscription_type'].value_counts()

subscription_type
Pay-as-you-go    5460
Monthly          4769
Annual           4730
Name: count, dtype: int64

In [63]:
customer_products = pd.merge(customer_transactions, products, on='product_id')
customer_products

,customer_id,join_date,age,income,country,customer_segment,subscription_type,churn_risk_score,transaction_id,product_id,...,shipping_cost,net_value,category,subcategory,price,manufacturing_cost,stock_level,supplier_id,reorder_point,rating
0,1,2023-01-01,40,87987.11,Canada,Premium,Annual,0.30,T366_1,7,...,15.20,3785.040,Clothing,Shirts,946.26,340.03,0,3,39,1.4
1,1,2023-01-01,40,87987.11,Canada,Premium,Annual,0.30,T366_2,29,...,10.24,1920.300,Electronics,Accessories,960.15,426.53,280,19,14,2.4
2,1,2023-01-01,40,87987.11,Canada,Premium,Annual,0.30,T366_3,41,...,16.93,2919.360,Home,Kitchen,973.12,7.41,28,18,21,1.3
3,1,2023-01-01,40,87987.11,Canada,Premium,Annual,0.30,T2523_1,194,...,11.28,642.520,Electronics,Phones,803.15,191.70,861,12,46,1.2
4,2,2023-01-02,33,78492.67,Canada,Basic,Annual,0.14,T1965_1,36,...,18.76,1831.380,Sports,Clothing,915.69,171.03,773,7,21,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14954,1000,2025-09-26,41,45101.95,UK,Standard,Monthly,0.95,T2604_1,185,...,18.88,482.416,Home,Kitchen,603.02,40.51,601,15,26,3.8
14955,1000,2025-09-26,41,45101.95,UK,Standard,Monthly,0.95,T2604_2,73,...,7.90,234.117,Clothing,Dresses,86.71,13.92,270,20,13,3.8
14956,1000,2025-09-26,41,45101.95,UK,Standard,Monthly,0.95,T2604_3,184,...,19.31,350.560,Sports,Clothing,175.28,64.97,153,9,40,2.7
14957,1000,2025-09-26,41,45101.95,UK,Standard,Monthly,0.95,T2604_4,7,...,6.20,3785.040,Clothing,Shirts,946.26,340.03,0,3,39,1.4


In [64]:
customer_products.groupby('category')['net_value'].mean()

category
Books          1103.943866
Clothing       1197.118322
Electronics    1139.283860
Home           1159.550420
Sports         1230.303327
Name: net_value, dtype: float64

In [38]:
marketing

,campaign_id,campaign_name,start_date,end_date,channel,budget,target_segment,conversion_rate
0,1,Campaign_1,2024-01-01,2024-02-01,Social Media,6811.37,Standard,0.142
1,2,Campaign_2,2024-01-02,2024-02-02,Display Ads,1625.54,Basic,0.070
2,3,Campaign_3,2024-01-03,2024-02-03,Search,2015.47,Premium,0.067
3,4,Campaign_4,2024-01-04,2024-02-04,Email,4046.47,Basic,0.099
4,5,Campaign_5,2024-01-05,2024-02-05,Email,7081.31,Basic,0.103
5,6,Campaign_6,2024-01-06,2024-02-06,Email,1238.04,Premium,0.072
6,7,Campaign_7,2024-01-07,2024-02-07,Social Media,7953.36,Premium,0.139
7,8,Campaign_8,2024-01-08,2024-02-08,Display Ads,2827.09,Premium,0.017
8,9,Campaign_9,2024-01-09,2024-02-09,Email,6035.68,Standard,0.116
9,10,Campaign_10,2024-01-10,2024-02-10,Email,2810.41,Premium,0.039


COMPLEX ANALYSIS EXERCISES

1. Customer Lifetime Value Analysis
   - Calculate CLV for each customer
   - Segment customers based on CLV
   - Analyze factors influencing CLV

2. Product Performance Analytics
   - Calculate product profitability
   - Analyze inventory turnover
   - Identify seasonal trends
   - Perform market basket analysis

3. Campaign ROI Analysis
   - Calculate campaign effectiveness
   - Analyze channel performance
   - Measure customer response rates

4. Churn Analysis
   - Predict customer churn
   - Identify churn factors
   - Create retention strategies

5. Advanced Visualizations
   - Create interactive dashboards
   - Generate automated reports
   - Build predictive models

ADDITIONAL EXERCISES:

1. Predictive Analytics
   - Build a churn prediction model
   - Forecast sales for next quarter
   - Predict inventory requirements

2. Market Basket Analysis
   - Identify frequently co-purchased items
   - Calculate support and confidence
   - Generate product recommendations

3. Customer Segmentation
   - Create RFM segments
   - Analyze segment behavior
   - Develop targeted strategies

4. Supply Chain Analytics
   - Optimize inventory levels
   - Analyze supplier performance
   - Calculate reorder points

5. Advanced Reporting
   - Create automated PDF reports
   - Build interactive dashboards
   - Generate email alerts
